In [238]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re
import os
import pdfplumber


## scrping av del 1

In [54]:
from bs4 import BeautifulSoup, NavigableString
import requests

def extract_text(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the divs with the specified classes
    divs = soup.find_all(class_=["RichtextArea ProductPage__richtext text-wrapper", "layout--2col wrapper"])

    # Extract the text of each child of each div until a h2, h3 or ul tag is encountered
    text = []
    for div in divs:
        for child in div.children:
            if isinstance(child, NavigableString):
                text.append(child.strip())
            elif child.name in ['ul']:
                break
            else:
                text.append(child.get_text(strip=True))
    text_string = ' '.join(text)
        
    return text_string
df = pd.read_csv("data/tags_30_11.csv")
number = 23

print(extract_text(df["url"][number]))

print(df["url"][number])

 EK80 Portable EntryThe EK80 Portable Entry is all you need for ecosystem monitoring and analysis in one package – at a reasonable price.  EK80 Portable Entry bundle comprises:   This is a complete rugged echo sounder with a built-in computer, GPS, and WiFi for remote operation. The core of the product is the WBT Mini, a compact EK80 wideband transceiver with four channels. The Entry bundle uses three of these channels for the 38 kHz split beam, while the last channel is used for the 200 kHz single beam.    The EK80 Portable entry is provided with the ES38-18/200-18 transducer. This is a dual-frequency transducer combining 38 and 200 kHz in one housing. The beamwidth is 18 degrees at both frequencies to secure a large sampling volume in shallow water. Combining high and low frequencies is ideal for mapping ecosystem components and separating fish from plankton.   Key features 
https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/sci_es_transceivers/ek80

## scraping av data sheets

In [241]:
def find_datasheets(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the div with the class "Downloads"
    downloads_div = soup.find('div', class_="Downloads")

    # If the div is found, find the section with the subtitle "Data sheet", "DATA SHEET", or "Data- and product sheets" within this div
    if downloads_div:
        datasheets_subtitle = downloads_div.find('h3', class_="Section__subtitle", string=re.compile("data sheet|data- and product sheets|Brochure", re.I))

        # If the subtitle is found, find the next sibling section
        if datasheets_subtitle:
            datasheets_section = datasheets_subtitle.find_next_sibling()

            # If the section is found, find all links within this section
            if datasheets_section:
                datasheet_links = datasheets_section.find_all('a', class_="Downloads__itemLink")

                # If the links are found, prepend "https://www.kongsberg.com" to each href attribute and return the list
                if datasheet_links:
                    links = [link.get('href') if 'https://simrad.online' in link.get('href') or 'https://www.simrad.online' in link.get('href') else "https://www.kongsberg.com" + link.get('href') for link in datasheet_links if link.get('href').endswith('.pdf')]
                    return ', '.join(links)
        else:
            # If no subtitle is found, find all links where the direct child span has the text "Datasheet"
            datasheet_links = downloads_div.find_all('a', class_="Downloads__itemLink")
    
            # If the links are found, return the href attribute of the first link that ends with ".pdf"
            for link in datasheet_links:
                href = link.get('href')
                if href.endswith('.pdf'):
                    return href if 'https://simrad.online' in href or 'https://www.simrad.online' in href else "https://www.kongsberg.com" + href
    # If the section, the div, or the links are not found, return None
    return None

In [242]:
df = pd.read_excel("data/12_04/tags_12_04.xlsx")

# Add a new column "Data sheets" to the dataframe
df['Data sheets'] = df['url'].apply(find_datasheets)

# only keep column url, product name and data sheets
df = df[['url', 'Product_Name', 'Data sheets']]
df.to_csv("data/data_sheets.csv", index=False)
df.to_excel("data/data_sheets.xlsx", index=False)

## Scraping av PDF

In [603]:
def scrape_pdf_text(url):
    print(url)
    # Download the PDF file
    response = requests.get(url)
    with open('temp.pdf', 'wb') as f:
        f.write(response.content)

    # Open the PDF file
    with pdfplumber.open('temp.pdf') as pdf:
        # Extract text from each page
        text = ''
        in_features_section = False
        if len(pdf.pages) > 10:
            return None
        for i, page in enumerate(pdf.pages):
            # If this is the last page, define a crop box that excludes the last 1 cm from the bottom
            if i == len(pdf.pages) - 1:
                crop_box = (0, 0, page.width, page.height - 70)
                page = page.crop(crop_box)

            page_text = page.extract_text().split('\n')
                        
            for line in page_text:
                if not in_features_section:
                    line = line.replace('®', '')  # Remove ® symbol
                    if line.startswith('•'):
                        text += '\n' + line  # Add bullet point to new line
                    else:
                        text += line + '\n'

    # Remove the temporary PDF file
    os.remove('temp.pdf')

    return text

In [255]:

# Load the CSV file
data_sheets = pd.read_csv("data/data_sheets.csv")
data_sheets = data_sheets.head(40)

# Filter rows with valid data sheet URLs
data_sheets = data_sheets[data_sheets['Data sheets'].notna()]
data_sheets = data_sheets[data_sheets['Data sheets'].str.count(',') < 1]

# Scrape the text from each data sheet and save it as a new column
data_sheets['Scraped Text'] = data_sheets['Data sheets'].apply(scrape_pdf_text)

# Save the DataFrame to a new CSV file
data_sheets.to_csv("data/data_sheets_with_text.csv", index=False)

In [577]:
%history

import requests
from bs4 import BeautifulSoup
def extract_text(url):
    # Get the HTML of the page
    response = requests.get(url)
    html = response.text

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all elements with the specified class
    elements = soup.find_all(class_="RichtextArea ProductPage__richtext text-wrapper")

    # Extract the text of each element
    rich_text = [element.get_text(strip=True) for element in elements]
    rich_text_string = ' '.join(rich_text)

    link = soup.find('a', href='#technicalInformation')

    bullet_points = []
    bullet_points_string = ""
    # If the link was found, find the element it links to
    if link is not None:
        target_id = link['href'].lstrip('#')
        target_element = soup.find(id=target_id)

        # If the target element was found, get its text
        if target_element is not None:
            # Find all the list items in the target element
            list_ite

## GPT formating

In [585]:
import streamlit as st
import pandas as pd
from openai._client import OpenAI

client = OpenAI(
    api_key=st.secrets["openai"]["api_key"],
)


In [586]:
system_prompt = """
                Read the whole file. Task: You are a content/format writer tasked with converting information from a PDF file into a a selected format. Only use the content found in the provided file. If there's insufficient information, insert "I don't know" in the respective block. Use the same sentences, same way of writing in the blocks if that is possible. Do not add any new information, and keep changes to a minimum, you are a formater, more than a content writer. Add where you found the information for each block.

                Write the following three content blocks:

                "General text":{ (200 - 400 words):
                Provide factual information blocks explaining the value that the products bring. Include a minimum of one and a maximum of three such blocks. If the product pdf offer more text, use three blocks. If the text has headings, use these headings. Have at elast 100 words in each block and add a suitable heading for each block.}

                "Key Features": { (40 - 100 words):
                List the key features or benefits of the product. Include as many as needed. Use bullet points for each feature. Use the same bullet points as the pdf.}

                "Technical Specifications": {(10 - 150 words):
                Add technical specifications related to the product, following the expected structure: "category, parameters, and parameter value." Include as many specifications as needed. Use bullet points for each specification.}



                Return as Json with the same formats as always."""

In [703]:
system_prompt_json_blocks = """
                Read the whole file. Task: You are a content/format writer tasked with converting information from a PDF file into a a selected format. Only use the content found in the provided file. If there's insufficient information, insert "I don't know" in the respective block. Use the same sentences, same way of writing in the blocks if that is possible. Do not add any new information, and keep changes to a minimum, you are a formater, more than a content writer. Include a minimum of one and a maximum of three such blocks. If the product pdf offer more text, use three blocks. If the text has headings, use these headings. Have at elast 100 words in each block and add a suitable heading for each block.

                Write the following three content blocks with (50 - 200 words) per block:


                "Block 1":{ 
                Use the information provided of the product to write a block of text that describes the product. The product often have a suiteable heading, use this heading and the text after. 
                }

                "Block 2":{
                Use the information provided of the product to write a block of text that describes the product. The product often have a suiteable heading, use this heading and the text after. 
                }

                "Block 3":{ 
                Use the information provided of the product to write a block of text that describes the product. The product often have a suiteable heading, use this heading and the text after. 
                }

                
             
                Return as Json with the same formats as always."""

In [702]:
def generate_blocks(text):
    messages = [
            {
                "role": "system",
                "content": system_prompt_json_blocks
            },
            {
                "role": "user",
                "content": f"I have a text that I want to format into headings and text bulks. The text is:\n\n{text}\n\n Please format this text using the same words and languange as the text. Do not change the text to much, and do not add information that is not there. return as json.
            }
        ]

    response = client.chat.completions.create(
        # model="gpt-3.5-turbo-1106",
        model="gpt-4-1106-preview",
        response_format={ "type": "json_object" },
        messages=messages,
    )
    output_text = response.choices[0].message.content.strip()
    return output_text

In [587]:
def generate_text(text):
    messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": f"I have a text that I want to format into headings and text bulks. The text is:\n\n{text}\n\n Please format this text using the same words and languange as the text. Do not change the text to much, and do not add information that is not there. return as json with key and value. Do not add any list or dictionary."
            }
        ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        response_format={ "type": "json_object" },
        messages=messages,
    )
    output_text = response.choices[0].message.content.strip()
    return output_text

In [610]:
def generate_text_general(prompt, text):
    messages = [
            {
                "role": "system",
                "content": "Help the user to the best of your ability. If you don't know the answer, say 'I don't know'."
            },
            {
                "role": "user",
                "content": f"{prompt}. Use this text {text}"
            }
        ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )
    output_text = response.choices[0].message.content.strip()
    return output_text

In [635]:
# file_melted = pd.read_csv("data/file_melted.csv")

for index, row in file_melted.iterrows():
    if row["Scraped Text"] != None:
        # if row["datasheet_name"] is empty
        if pd.isnull(row["General text"]):
            print(index,"product_name:", row["Product_Name"])


            input_text = row["Scraped Text"]
            MAX_TOKENS = 16385  # Maximum number of tokens


            # Truncate the text if it's too long
            if len(input_text.split()) > MAX_TOKENS:
                input_text = ' '.join(input_text.split()[:MAX_TOKENS])


            data_sheet_name = generate_text_general("From the text, return a single name that work as a product name and header. Onøy return this name.",input_text)
            file_melted.loc[index, 'datasheet_name'] = data_sheet_name
            print(data_sheet_name, "-----",row["Data sheet"])
            output_text = generate_text(input_text)
            print(output_text)
            # Parse the JSON data
            data = json.loads(output_text)
            data = {k.lower(): v for k, v in data.items()}


            block1 = data['general text']
            block2 = data['key features']
            block3 = data['technical specifications']
            print(block1)
            print(block2)
            print(block3)
            # Add the blocks to the DataFrame
            file_melted.loc[index, 'General text'] = str(block1)
            file_melted.loc[index, 'Features'] = str(block2)
            file_melted.loc[index, 'Technical Specifications'] = str(block3)
file_melted.to_csv("data/test.csv", index=False)
file_melted.to_excel("data/test.xlsx", index=False)

In [ ]:
_ih[10-40:] # code of the 10 most recently run cells (Even if those cells are deleted now)*


### MERGE AND MAKE FILEs

In [591]:
import pandas as pd

df = pd.read_excel('data/multiple_datasheets_v1.xlsx')



# List of all the data sheet columns
data_sheet_columns = [f'Data sheet {i}' for i in range(1, 17)]

df_melted = df.melt(id_vars=['url', 'Product_Name'], value_vars=data_sheet_columns, var_name='DataSheet', value_name='Value')

#remove NA
df_melted = df_melted[df_melted['Value'].notna()]

df_melted.to_excel('file_melted.xlsx', index=False)

In [ ]:
file_melted = pd.read_excel("file_melted.xlsx")
# remove whitespace
file_melted['Data sheet'] = file_melted['Data sheet'].str.strip()
# remove duplicate data sheets
file_melted = file_melted.drop_duplicates(subset=['Data sheet'])


file_melted['Scraped Text'] = file_melted['Data sheet'].apply(scrape_pdf_text)
file_melted.to_csv("data/file_melted.csv", index=False)


In [639]:
all_products_categories = pd.read_excel("data/all_products_category.xlsx", usecols=["Product_Name", "Product category"])

all_products_ai = pd.read_excel("data/styrings_data/alle_produkter_ai_test.xlsx")

# merge on product name and product category
all_products_ai_category = pd.merge(all_products_ai, all_products_categories, on=["Product_Name"], how="left")

print(all_products_ai_category.columns)
all_products_ai_category.to_excel("data/all_products_ai_category.xlsx", index=False)
all_products_ai_category.to_csv("data/all_products_ai_category.csv", index=False)

Index(['Product_Name', 'Data sheets', 'Features', 'General text',
       'Technical Specifications', 'url', 'Product category'],
      dtype='object')


In [646]:
all_products_categories = pd.read_excel("data/all_products_category.xlsx", usecols=["Product_Name", "Product category"])
multiple_datasheets_categories = pd.read_csv("data/test.csv")

# merge on product name and product category
multiple_datasheets_categories = pd.merge(multiple_datasheets_categories, all_products_categories, on=["Product_Name"], how="left")

multiple_datasheets_categories.to_excel("data/multiple_datasheets_categories.xlsx")
multiple_datasheets_categories.to_csv("data/multiple_datasheets_categories.csv", index=False)



In [644]:
multiple_datasheets_categories = pd.read_csv("data/test.csv")
multiple_datasheets_categories.to_excel("data/multiple_datasheets_AI.xlsx", index=False)

In [660]:
multiple_datasheets_categories = pd.read_excel("data/multiple_datasheets_categories.xlsx")


In [663]:
for i, row in multiple_datasheets_categories.iterrows():
    match = re.search(r'"(.*?)"', row["datasheet_name"])
    if match:
        value = match.group(1)
        multiple_datasheets_categories.loc[i, "datasheet_name"] = value
        print(value," from : ", row["datasheet_name"])

# remove duplicate data sheets
multiple_datasheets_categories = multiple_datasheets_categories.drop_duplicates(subset=['Scraped Text'])
multiple_datasheets_categories.to_excel("data/multiple_datasheets_categories_newname.xlsx", index=False)

In [672]:
multiple_datasheets_categories = pd.read_excel("data/multiple_datasheets_categories_newname.xlsx")

all_products = pd.read_excel("data/styrings_data/12-12-all_products.xlsx")

# merge on product name and product category
all_products = pd.concat([all_products, multiple_datasheets_categories], axis=0, ignore_index=True)


all_products.to_excel("data/all_products.xlsx", index=False)

In [682]:
multiple_datasheets_names = pd.read_excel("data/multiple_datasheets_v1.xlsx", usecols=["Product_Name"])

print(len(multiple_datasheets_names.values))

all_products_names = pd.read_excel("data/all_products.xlsx")
# remove the products that are in multiple datasheets


# Store original Product_Name values
original_product_names = all_products_names['Product_Name']

# Perform the operation
all_products_names = all_products_names[~all_products_names['Product_Name'].isin(multiple_datasheets_names['Product_Name'])]

# Find and print the removed names
removed_names = original_product_names[~original_product_names.isin(all_products_names['Product_Name'])]
print(removed_names)
print(len(removed_names.values))
all_products_names.to_excel("data/merged_multiple_data_frames_without_names.xlsx", index=False)


36
0                            AIS physical shore stations
2                                    AIS space receivers
5              Acoustic control system for BOP operation
8                   Automatic Identification System, AIS
9                   Autonomous Underwater Vehicle, HUGIN
15        DGPS / DGNSS reference and monitoring stations
19                     EA440 - shallow and medium waters
25            EM 124 Multibeam echosounder, Max. 11000 m
30                     EM 304 MKII Multibeam echosounder
32             EM 712 Multibeam echosounder, Max. 3600 m
33                        Echo sounder sweep system, MCU
36                      GNSS compass and position sensor
37                   GNSS-based Relative Position Sensor
38                           GNSS/DGNSS Position Sensors
40                   GPS transponder, Seatrack 220 / 320
41                         GPS transponder, Seatrack 330
44                            Helideck Monitoring System
57                  Laser-ba

In [ ]:
# Iterate through the URLs
for url in urls:
    # Check if the URL contains the text "<span class="ProductPage__discontinuedBanner"> No longer sold </span>"
    if '<span class="ProductPage__discontinuedBanner"> No longer sold </span>' in url:
        # Remove the URL from the list
        urls.remove(url)


In [686]:
import requests
from bs4 import BeautifulSoup

# Define a function to check if the URL contains the specified text
def check_url(url):
    if isinstance(url, str):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = soup.find('span', class_='ProductPage__discontinuedBanner')
        if result and "No longer sold" in result.text.strip():
            print(url)
            return False
        else:
            return True
    else:
        return True

# Apply the function to the 'url' column and keep only the rows where the function returns True
all_products = pd.read_excel("data/all_products.xlsx")
all_products = all_products[all_products['url'].apply(check_url)]

https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/multibeam-echo-sounders/em-304-multibeam-echosounder-max.-8000-m
https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/echosounder-sweep-system/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA400-600/
https://www.kongsberg.com/maritime/products/commercial-fisheries/fisherysonar/cs90/
https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/simrad-ek15/
https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/simrad-ek60/
https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-120khz-transducers/simrad-es120-7dd/
https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-38khz-transducers/es38_12/
https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-38khz-transducers/es38b/
https://www.kongsberg.com/maritime/products/commercial-fisheries/fisherysounder/es70/
https://www.kongsberg.com/maritime/products/commercial-fisheries/simrad-trawl-sonars/fm90/
https://www.kongsberg.com/maritime/products/commercial-fisheries/cm-sensors/pi-configurator/
https://www.kongsberg.com/mariti

In [697]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define a function to check if the URL contains the specified text
def check_url(url):
    if isinstance(url, str):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = soup.find('span', class_='ProductPage__discontinuedBanner')
        if result and "No longer sold" in result.text.strip():
            return False
    return True

# Read the DataFrame
all_products = pd.read_excel("data/all_products_manually_done.xlsx")

# Create a new DataFrame to store the rows to keep
products_to_keep = []
# Define a list to store the indices of the rows to remove
indices_to_remove = []

# Iterate over the rows of the DataFrame
for index, row in all_products.iterrows():
    # If the URL does not contain the specified text
    if check_url(row['url']):
        # add one count for evry produt that is true
        products_to_keep.append(str(row['url'])+" :"+str(row['Product_Name']))
    else:
        print("remove this",row['url'], " : ", row['Product_Name'])
        indices_to_remove.append(index)

        # Iterate over the rows of the DataFrame
        for index2, row2 in all_products.iterrows():
            if row["Product_Name"] == row2["product_family_name"]:
                print("family grouped also removed: ",row2["url"], " : ", row2["Product_Name"])
                indices_to_remove.append(index2)

# Remove the rows with the collected indices
all_products = all_products.drop(indices_to_remove)



remove this https://www.kongsberg.com/maritime/products/Acoustics-Positioning-and-Communication/transponders/cnode-transponders-for-hipap-and-uPAP/cNODE-IQAM/  :  cNODE IQAM - Intelligent data analysis and monitoring


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA-600/  :  EA 400/600 Sidescan
remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA-600/  :  EA 600 Series
remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/multibeam-echo-sounders/em-304-multibeam-echosounder-max.-8000-m  :  EM 304 Multibeam echosounder, Max. 8000 m


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA-600/  :  Remote display RD301


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA400-600/  :  Side looking transducer 200 kHz
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/fisherysonar/cs90/  :  Simrad CS90
remove this https://www.kongsberg.com/maritime/products/ocean-science/ocean-science/es_scientific/simrad-ek15/  :  Simrad EK15


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-120khz-transducers/simrad-es120-7dd/  :  Simrad ES120-7DD
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-38khz-transducers/es38_12/  :  Simrad ES38-12
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/td/simrad-38khz-transducers/es38b/  :  Simrad ES38B
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/fisherysounder/es70/  :  Simrad ES70
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/simrad-trawl-sonars/fm90/  :  Simrad FM90
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/cm-sensors/pi-configurator/  :  Simrad PI Configurator
remove this https://www.kongsberg.com/maritime/products/ocean-science/mapping-systems/es_bottommapping/EA400-600/  :  EA400SP
remove this https://www.kongsberg.com/maritime/products/commercial-fisheries/fisherysonar/sh90/  :  Si

In [704]:
all_products.to_excel("data/all_products_dropped_discontinuid.xlsx", index=False)


,Product_Name,Product category,General text,Features,Technical Specifications,url,Data sheets,downloads,product_family_name,is_range
0,"Seapath 380 - utilising GPS, GLONASS, Galileo...",Seafloor mapping,The Seapath 380 series uses a state-of-the-art...,• 0.007° to 0.02° roll and pitch accuracy depe...,PERFORMANCE POWER SPECIFICATIONS\nHeave accura...,NaN,https://www.kongsberg.com/contentassets/c22a59...,{'Data sheet': ['https://www.kongsberg.com/con...,NaN,NaN
1,12 kHz dual-beam transducer,"Seafloor mapping, Geophysical survey",The Simrad 12-16/60 is a 12 kHz dual beam tran...,Resonant frequency: 12 kHz\nDual beam transduc...,Resonant frequency: 12 kHz\nMaximum transducer...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0
2,120 kHz single-beam transducer,"Seafloor mapping, Geophysical survey",Introduction The Simrad 120-25 is a single-bea...,"Resilient Design: Housed in a robust casing, t...",Resonant frequency: 120 kHz Maximum input powe...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0
3,18 kHz single-beam transducer,"Seafloor mapping, Geophysical survey","The Simrad 18-11 is a large, single-beam trans...",Key Features:\n- Resonant frequency: 18 kHz\n-...,Technical Specifications:\n- Resonant frequenc...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0
